In [262]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
from collections import defaultdict
import re

In [263]:
stations_df = pd.read_csv('stations_zip.csv')
stations_df.head()

,Unnamed: 0,NAME,LINE,LATITUDE,LONGITUDE,zipcode
0,0,103rd St,ABC,40.796061,-73.961370,10025-4403
1,1,103rd St,466,40.790600,-73.947478,10029
2,2,103rd St,1,40.799446,-73.968379,10025
3,3,103rd St - Corona Plaza,7,40.749865,-73.862700,11368
4,4,104th St,AS,40.681711,-73.837683,11419


In [264]:
stations_df = pd.read_csv('stations_zip.csv')
stations_df.drop(['LATITUDE', 
                  'LONGITUDE'], axis=1, inplace = True)
for column in stations_df.columns:    
    stations_df.rename(columns={column: column.lower().strip()}, inplace=True)

    
stations_df.sort_values(by = ['name'], inplace = True)

In [265]:
def get_data(week_nums):
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
    return pd.concat(list(map(lambda x: pd.read_csv(url.format(x)), week_nums)))
weeks = [200606, 200530, 200523, 200516, 200509, 200502, 200425, 200418, 200411]
turnstile_df = get_data(weeks)


In [266]:
turnstile_df.sort_values(by = ['STATION'], inplace = True)
turnstile_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
32587,H008,R248,01-00-01,1 AV,L,BMT,04/23/2020,08:56:30,REGULAR,2803485,16383318
33095,H007A,R248,02-03-02,1 AV,L,BMT,05/25/2020,04:00:00,REGULAR,101109,177624
33094,H007A,R248,02-03-02,1 AV,L,BMT,05/25/2020,00:00:00,REGULAR,101108,177622
33093,H007A,R248,02-03-02,1 AV,L,BMT,05/24/2020,20:00:00,REGULAR,101100,177615
33092,H007A,R248,02-03-02,1 AV,L,BMT,05/24/2020,16:00:00,REGULAR,101085,177596


***Here's how we're solving it:***  
We create this dictionary, and then construct the same dictionary for the turnstile data.  
Then we run simple comparisons to match up by stop_name and line (the subway line), and then in the stations dictionary, append a field with the turnstyle ID associated with the station ID. Then we simply need to apply that dictionary and it's zipcodes to the turnstile dataframe. AND THEN IT'S LITERALLY DONE.  
We will have the turnstile data with zip-codes!

In [267]:
stations_df.name = stations_df.name.str.replace('rd', '')
stations_df.name = stations_df.name.str.replace('th', '')
stations_df.name = stations_df.name.str.replace('nd', '')


In [268]:
stations_dict = defaultdict(dict)

for i in stations_df.itertuples():
    stations_dict[i[1]]['stop_name'] = re.sub('[^A-Za-z0-9]+', '', i[2]).lower()
    stations_dict[i[1]]['line'] = i[3]
    stations_dict[i[1]]['zipcode'] = i[4]
    stations_dict[i[1]]['turnstile_station_id'] = ''
    

In [269]:
stations_dict

defaultdict(dict,
            {0: {'stop_name': '103st',
              'line': 'ABC',
              'zipcode': '10025-4403',
              'turnstile_station_id': ''},
             1: {'stop_name': '103st',
              'line': '466',
              'zipcode': '10029',
              'turnstile_station_id': ''},
             2: {'stop_name': '103st',
              'line': '1',
              'zipcode': '10025',
              'turnstile_station_id': ''},
             3: {'stop_name': '103stcoronaplaza',
              'line': '7',
              'zipcode': '11368',
              'turnstile_station_id': ''},
             4: {'stop_name': '104st',
              'line': 'AS',
              'zipcode': '11419',
              'turnstile_station_id': ''},
             5: {'stop_name': '104102sts',
              'line': 'JZ',
              'zipcode': '11418',
              'turnstile_station_id': ''},
             6: {'stop_name': '110st',
              'line': '466',
              'zipcode': '1002

In [270]:
turnstiles_dict = defaultdict(dict)

for i in turnstile_df.itertuples():
    turnstiles_dict[i[2]]['stop_name'] = re.sub('[^A-Za-z0-9]+', '',i[4]).lower()
    turnstiles_dict[i[2]]['line'] = i[5]

In [259]:
x = 'w4stwashsq'
y = 'w4stwashingtonsqupper'
print(fuzz.ratio(x, y))
print(x in y)
print(y in x)

65
False
False


In [271]:
turnstiles_dict

defaultdict(dict,
            {'R248': {'stop_name': '1av', 'line': 'L'},
             'R180': {'stop_name': '103st', 'line': '6'},
             'R314': {'stop_name': '103st', 'line': 'BC'},
             'R191': {'stop_name': '103st', 'line': '1'},
             'R208': {'stop_name': '103stcorona', 'line': '7'},
             'R007': {'stop_name': '104st', 'line': 'JZ'},
             'R354': {'stop_name': '104st', 'line': 'A'},
             'R181': {'stop_name': '110st', 'line': '6'},
             'R008': {'stop_name': '111st', 'line': 'J'},
             'R355': {'stop_name': '111st', 'line': 'A'},
             'R310': {'stop_name': '111st', 'line': '7'},
             'R333': {'stop_name': '116st', 'line': 'BC'},
             'R182': {'stop_name': '116st', 'line': '6'},
             'R324': {'stop_name': '116st', 'line': '23'},
             'R159': {'stop_name': '116stcolumbia', 'line': '1'},
             'R009': {'stop_name': '121st', 'line': 'JZ'},
             'R132': {'stop_name': '1

In [272]:
for i in turnstiles_dict:
    for j in stations_dict:
        is_name_match = True
        if turnstiles_dict[i]['stop_name'] == stations_dict[j]['stop_name']:
            is_name_match = True
        elif turnstiles_dict[i]['stop_name'] in stations_dict[j]['stop_name']:
            is_name_match = True
        elif stations_dict[j]['stop_name'] in turnstiles_dict[i]['stop_name']:
            is_name_match = True
        elif fuzz.ratio(turnstiles_dict[i]['stop_name'], stations_dict[j]['stop_name']) >= 65:
            is_name_match = True
        else:
            is_name_match = False
            
        if is_name_match:
            if len(stations_dict[j]['line']) <= len(turnstiles_dict[i]['line']):
                station_line_match = True
                for char in list(stations_dict[j]['line']):
                    if char not in turnstiles_dict[i]['line']:
                        statioin_line_match = False
                        break
            elif len(stations_dict[j]['line']) >= len(turnstiles_dict[i]['line']):
                turnstile_line_match = True
                for char in list(turnstiles_dict[i]['line']):
                    if char not in stations_dict[j]['line']:
                        turnstile_line_match = False
                        break
            if station_line_match or turnstile_line_match:
                stations_dict[j]['turnstile_station_id'] = i

                                
                    
            
            
                            
            

In [273]:
count = 0
total = 0
for i in stations_dict:
    if stations_dict[i]['turnstile_station_id'] == '':
        count+=1
    total+=1
print(count,total)

10 470
